In [1]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

LANG_CHAIN_API_KEY = os.getenv('LANG_CHAIN_API_KEY')
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
JINA_EMBEDDING_KEY = os.getenv('JINA_EMBEDDING_KEY')


In [2]:

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    api_key=GEMINI_API_KEY,
    temperature=0
)

c:\Users\DELL\anaconda3\envs\chainlit1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
! pip freeze > requirements.txt

In [4]:
#from langchain.document_loaders import PyPDFLoader
#from langchain_core.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    # Duplicate documents on purpose - messy data
    PyPDFLoader("documents/temp.pdf"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())


In [5]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [6]:
splits = text_splitter.split_documents(docs)
len(splits)

5

In [7]:
from langchain.embeddings import JinaEmbeddings
embedding = JinaEmbeddings(jina_api_key = JINA_EMBEDDING_KEY, model_name="jina-embeddings-v2-base-en")


In [8]:
from langchain.vectorstores import Chroma
# persist_directory = 'docs/chroma1/'
persist_directory = 'chroma4/'
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

⚠️ It looks like you upgraded from a version below 0.6 and could benefit from vacuuming your database. Run chromadb utils vacuum --help for more information.


In [9]:
# ! pip install chromadb

In [10]:
print(vectordb._collection.count())

232


In [11]:
question = "is there an email i can ask for help"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [12]:
vectordb.persist()

C:\Users\DELL\AppData\Local\Temp\ipykernel_9200\3711397106.py:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [13]:
# import datetime
# current_date = datetime.datetime.now().date()
# if current_date < datetime.date(2023, 9, 2):
#     llm_name = "gpt-3.5-turbo-0301"
# else:
#     llm_name = "gpt-3.5-turbo"
# print(llm_name)

In [14]:
# from langchain.chat_models import ChatOpenAI
# llm = ChatOpenAI(model_name=llm_name, temperature=0)

In [15]:
from langchain.chains import RetrievalQA

In [24]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)
from langchain.prompts import PromptTemplate

print(qa_chain)

question="how many words are there?"
result = qa_chain({"query": question})
result['result']

verbose=False combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="Use the following pieces of context to answer the user's question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})]), llm=ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), temperature=0.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002937A091E50>, default_metadata=(

'I cannot count the number of words because the provided text is incomplete (indicated by "Page 1 of 3" and "Page 2 of 3").  To accurately count the words, I need the full text.\n'

In [17]:
# from langchain.prompts import PromptTemplate

# # Build prompt
# template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
# {context}
# Question: {question}
# Helpful Answer:"""
# QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [18]:
# # Run chain
# qa_chain = RetrievalQA.from_chain_type(
#     llm,
#     retriever=vectordb.as_retriever(),
#     return_source_documents=True,
#     chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
# )

In [19]:

# Build prompt
from langchain.prompts import PromptTemplate

# template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "Fuck you !" at the end of the answer. 
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Must say "Fuck You!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)

# Run chain
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

question = "Is probability a class topic?"
result = qa_chain({"query": question})
result["result"]


# ### Memory

from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


# ### ConversationalRetrievalChain


from langchain.chains import ConversationalRetrievalChain

retriever = vectordb.as_retriever()
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)


C:\Users\DELL\AppData\Local\Temp\ipykernel_9200\720636281.py:25: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})
C:\Users\DELL\AppData\Local\Temp\ipykernel_9200\720636281.py:33: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [20]:
chat_history = []
answer = ""
db_query = ""
db_response = []

In [21]:
query ="What is total experience mentioned?"

result = qa({"question": query, "chat_history": chat_history})
chat_history.extend([(query, result["answer"])])
# db_query = result["generated_question"]
# db_response = result["source_documents"]
answer = result["answer"]
print(answer)

The provided text mentions a 2-month internship at NESCOM from July 2019 to August 2019 and  a separate experience providing ICT services to corporate customers, but the duration of that experience is not specified.  Therefore, I cannot calculate a total experience.

